<a href="https://colab.research.google.com/github/Anu-kriti/AA/blob/master/Gala_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from numpy import load
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')


import os
import numpy as np
import pandas as pd
import cv2
from zipfile import ZipFile 

file_name = '9d34462453e311ea.zip'

with ZipFile(file_name, 'r') as zip:
    zip.extractall()


In [6]:
path = 'dataset'

train_df = pd.read_csv(path+'/train.csv')
test_df = pd.read_csv(path+'/test.csv')
train_df.Class.value_counts(normalize = True)

Food                    0.380745
Attire                  0.282634
misc                    0.212435
Decorationandsignage    0.124185
Name: Class, dtype: float64

In [7]:
class_map = {
    'Food': 0,
    'Attire': 1,
    'Decorationandsignage': 2,
    'misc': 3
}

inverse_class_map = {
    0: 'Food',
    1: 'Attire',
    2: 'Decorationandsignage',
    3: 'misc'
}
train_df['Class'] = train_df['Class'].map(class_map).astype(np.uint8)
train_df.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [0]:
train_path = 'dataset/Train Images/'
test_path = 'dataset/Test Images/'

h, w = 224, 224

train_images, train_labels = [], []

for i in range(len(train_df.Image)):
    train_image = cv2.imread(train_path + str(train_df.Image[i]))
    train_image = cv2.resize(train_image, (h, w))
    train_images.append(train_image)
    train_labels.append(train_df.Class[i])

test_images = []

for i in range(len(test_df.Image)):
    test_image = cv2.imread(test_path + str(test_df.Image[i]))
    test_image = cv2.resize(test_image, (h, w))
    test_images.append(test_image)

train_images = np.array(train_images).astype('float32')
#train_images = train_images.astype('float32')/255.0
test_images = np.array(test_images).astype('float32')

In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(train_images, to_categorical(train_labels), test_size=0.2, random_state=42)

In [10]:
X_train.shape,y_train.shape

((4786, 224, 224, 3), (4786, 4))

In [11]:
import tensorflow as tf
tf.test.gpu_device_name()

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1152244118631715023, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12122589873734908250
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3707463848756320324
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7470045594
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16168383393346113427
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]

In [12]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(h, w, 3), 
    pooling='avg'
)

base_model.trainable = False

model = Sequential([
  base_model,
  #Dropout(0.2),
    Dense(512, activation = "relu"),
  Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

batch_size = 32
epochs = 10

datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True, 
            height_shift_range = 0.1,
            width_shift_range = 0.1,
            rotation_range = 10)


model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data= (X_test, y_test),
                    steps_per_epoch = len(X_train) / batch_size, epochs=epochs)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2052      
Total params: 24,638,852
Trainable params: 1,051,140
Non-trainable params: 23,587,712
_________________________________________________________________
Epoch 1/10
150/149 [==============================] - 63s 420ms/step - loss: 0.8520 - acc: 0.6943 - val_loss: 0.7323 - val_acc: 0.7143
Epoch 2/10
150/149 [==============================] - 53s 353ms/step - loss: 0.6141 -

In [13]:
labels = model.predict(test_images)
label = [np.argmax(i) for i in labels]
class_label = [inverse_class_map[x] for x in label]
print(class_label[:3])

model.save_weights('./resnet_1itr_model.h5', overwrite=True)

from numpy import savetxt
savetxt('resnet_1itr_prob.csv', labels, delimiter=',')

submission = pd.DataFrame({'Image': test_df.Image, 'Class': class_label })
submission.head()

submission.Class.value_counts()
submission.to_csv('resnet_1stitr_results.csv', index=False)

['Attire', 'Food', 'misc']


In [14]:
submission.Class.value_counts()

Food                    1160
Attire                  1033
misc                     630
Decorationandsignage     396
Name: Class, dtype: int64

In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape= X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape= X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(4))
  model.add(tf.keras.layers.Activation('softmax'))
  return model


In [18]:
import os

#with strategy.scope():
model = create_model()
#model.compile(
 #     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
  #    loss='sparse_categorical_crossentropy',
   #   metrics=['sparse_categorical_accuracy'])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 224, 224, 64)      4864      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 112, 112, 64)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     204928    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 128)      

In [19]:
model.fit(
    X_train, y_train,
    epochs=5,
    batch_size = 32,
   # steps_per_epoch= 12
    validation_data=(X_test, y_test)
    #validation_freq=17
)

Train on 4786 samples, validate on 1197 samples
Epoch 1/5
4786/4786 [==============================] - 49s 10ms/sample - loss: 16.5710 - acc: 0.2860 - val_loss: 1.3339 - val_acc: 0.3851
Epoch 2/5
4786/4786 [==============================] - 46s 10ms/sample - loss: 1.7979 - acc: 0.3055 - val_loss: 1.3357 - val_acc: 0.3885
Epoch 3/5
4786/4786 [==============================] - 46s 10ms/sample - loss: 1.7614 - acc: 0.2944 - val_loss: 1.3268 - val_acc: 0.3835
Epoch 4/5
4786/4786 [==============================] - 46s 10ms/sample - loss: 1.7116 - acc: 0.2965 - val_loss: 1.3242 - val_acc: 0.3835
Epoch 5/5
4786/4786 [==============================] - 46s 10ms/sample - loss: 1.6308 - acc: 0.3201 - val_loss: 1.3209 - val_acc: 0.3835


In [20]:
labels = model.predict(test_images)
label = [np.argmax(i) for i in labels]
class_label = [inverse_class_map[x] for x in label]
print(class_label[:3])

model.save_weights('./basic_model_1itr_model.h5', overwrite=True)

from numpy import savetxt
savetxt('basic_model_1itr_prob.csv', labels, delimiter=',')

submission = pd.DataFrame({'Image': test_df.Image, 'Class': class_label })
submission.head()

submission.Class.value_counts()
submission.to_csv('basic_model_1stitr_results.csv', index=False)

['Food', 'Food', 'Food']


In [21]:
submission.Class.value_counts()

Food      3206
Attire       8
misc         5
Name: Class, dtype: int64

In [12]:
!pip install efficientnet

In [13]:
import math, re, os
import numpy as np
import skimage
from matplotlib import pyplot as plt
#from kaggle_datasets import KaggleDatasets
import efficientnet  as efn
import tensorflow as tf, tensorflow.keras.backend as K
from tensorflow.keras.applications import DenseNet201
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.applications import ResNet152V2
print("Tensorflow version " + tf.__version__)

Tensorflow version 1.15.0


In [14]:
#pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = efn.EfficientNetB7(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
pretrained_model = DenseNet201(weights='imagenet', include_top=False ,input_shape=[224,224, 3])
#pretrained_model.trainable = False # tramsfer learning
pretrained_model.trainable = True
model_1 = tf.keras.Sequential([
        pretrained_model,
        #tf.keras.layers.Conv2D(filters=1024,kernel_size=3,strides=(2,2),padding='same'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation = "relu"),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
model_1.compile(
    optimizer = 'adam',
    #optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
    loss = 'categorical_crossentropy',
    metrics=['acc']
)
model_1.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
74842112/74836368 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 7, 7, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               983552    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2052      
Total params: 19,307,588
Trainable params: 19,078,532
Non-trainable params: 229,056
_________________________________________________________________


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
epochs = 5

datagen = ImageDataGenerator(
            rescale            = 1./255,
            shear_range        = 0.2,
            zoom_range         = 0.2,
            horizontal_flip    = True, 
            height_shift_range = 0.1,
            width_shift_range  = 0.1,
            rotation_range     = 10)


model_1.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data= (X_test, y_test),
                    steps_per_epoch = len(X_train) / batch_size, epochs=epochs)

Epoch 1/5
150/149 [==============================] - 1214s 8s/step - loss: 0.9472 - acc: 0.6162 - val_loss: 70.1739 - val_acc: 0.1587
Epoch 2/5
150/149 [==============================] - 1198s 8s/step - loss: 0.8999 - acc: 0.6421 - val_loss: 675.4382 - val_acc: 0.1236
Epoch 3/5
150/149 [==============================] - 1155s 8s/step - loss: 0.8461 - acc: 0.6578 - val_loss: 724.2254 - val_acc: 0.1195
Epoch 4/5
150/149 [==============================] - 1160s 8s/step - loss: 0.8214 - acc: 0.6720 - val_loss: 485.3948 - val_acc: 0.1270
Epoch 5/5
 58/149 [==========>...................] - ETA: 11:30 - loss: 0.7895 - acc: 0.6853

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os 
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)

tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
strategy


In [0]:
LABEL_NAMES = ['Food', 'Attire', 'Decorationandsignage', 'misc']
